# Single Player Feature Research Notebook

Research and build features for a specific player using backtest workflow.

**Workflow (mirrors backtest process):**
1. Load DFS salaries for target date (the "slate")
2. Select a player from the slate
3. Load historical data for that player (excluding target date)
4. Build features using only historical data
5. Analyze and visualize features

This ensures feature engineering matches production backtest logic.

## Setup

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

repo_root = Path.cwd().parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from src.data.storage.sqlite_storage import SQLiteStorage
from src.data.loaders.historical_loader import HistoricalDataLoader
from src.utils.fantasy_points import calculate_dk_fantasy_points
from src.features.pipeline import FeaturePipeline
from src.utils.feature_config import load_feature_config

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

print('Setup complete')

## Configuration

Set the target date and number of seasons to load.

**NUM_SEASONS**: 
- `1` = Current season only (from Oct 1 of season year)
- `2` = Current + previous season (default, ~16 months of data)
- `3` = Current + 2 previous seasons, etc.

In [13]:
TARGET_DATE = '20250210'
NUM_SEASONS = 2  # Current season + previous season

DB_PATH = repo_root / 'nba_dfs.db'

storage = SQLiteStorage(str(DB_PATH))
loader = HistoricalDataLoader(storage)

print(f'Target Slate Date: {TARGET_DATE}')
print(f'Seasons to load: {NUM_SEASONS} (current + previous)')
print(f'Database: {DB_PATH}')

Target Slate Date: 20250210
Seasons to load: 2 (current + previous)
Database: c:\Users\antho\OneDrive\Documents\Repositories\delapan-fantasy\nba_dfs.db


## Step 1: Load Slate Data (DFS Salaries)

This is the first step in the backtest process - load salaries for the target date.

In [14]:
print(f'Loading slate data for {TARGET_DATE}...')

slate_data = loader.load_slate_data(TARGET_DATE)

salaries_df = slate_data.get('dfs_salaries', pd.DataFrame())

if salaries_df.empty:
    print(f'ERROR: No salary data found for {TARGET_DATE}')
    print('\nTry a different date. Available dates:')
    
    available_dates = loader.load_slate_dates('20250101', '20250331')
    print(available_dates[:20])
else:
    print(f'\nSlate loaded successfully!')
    print(f'Players on slate: {len(salaries_df)}')
    
    if 'longName' in salaries_df.columns and 'playerName' not in salaries_df.columns:
        salaries_df['playerName'] = salaries_df['longName']
    
    print(f'\nSlate summary:')
    if 'salary' in salaries_df.columns:
        salaries_df['salary'] = pd.to_numeric(salaries_df['salary'], errors='coerce')
        print(f'  Average salary: ${salaries_df["salary"].mean():,.0f}')
        print(f'  Min salary: ${salaries_df["salary"].min():,.0f}')
        print(f'  Max salary: ${salaries_df["salary"].max():,.0f}')
    
    if 'pos' in salaries_df.columns:
        print(f'\nPlayers by position:')
        print(salaries_df['pos'].value_counts())
    
    print(f'\nTop 20 highest salaries on slate:')
    display_cols = ['playerName', 'team', 'pos', 'salary']
    display_cols = [col for col in display_cols if col in salaries_df.columns]
    
    if display_cols:
        top_salaries = salaries_df.nlargest(20, 'salary')[display_cols]
        display(top_salaries)

Loading slate data for 20250210...



Slate loaded successfully!
Players on slate: 342

Slate summary:
  Average salary: $4,623
  Min salary: $3,000
  Max salary: $12,700

Players by position:
pos
SF    84
SG    83
PG    60
PF    59
C     56
Name: count, dtype: int64

Top 20 highest salaries on slate:


,playerName,team,pos,salary
0,Nikola Jokic,DEN,C,12700
1,Giannis Antetokounmpo,MIL,PF,11700
2,Luka Doncic,LAL,PG,11000
3,Victor Wembanyama,SA,C,11000
4,Shai Gilgeous-Alexander,OKC,SG,10800
5,LeBron James,LAL,SF,10700
6,Anthony Davis,DAL,C,10500
7,Jayson Tatum,BOS,PF,10000
8,LaMelo Ball,CHA,PG,9900
9,Trae Young,ATL,PG,9800


## Step 2: Select Player from Slate

Choose a player to analyze. You can select by:
- Index from the table above
- Player name (partial match)

In [15]:
PLAYER_SELECTION = 'Nikola Jokic'

if not salaries_df.empty:
    if isinstance(PLAYER_SELECTION, int):
        selected_player = salaries_df.iloc[PLAYER_SELECTION]
    else:
        matches = salaries_df[
            salaries_df['playerName'].str.contains(PLAYER_SELECTION, case=False, na=False)
        ]
        
        if matches.empty:
            print(f'ERROR: No player matching "{PLAYER_SELECTION}" found on slate')
            print('\nAvailable players:')
            print(salaries_df['playerName'].tolist())
            selected_player = None
        elif len(matches) > 1:
            print(f'Multiple players match "{PLAYER_SELECTION}":')
            display(matches[['playerName', 'team', 'pos', 'salary']])
            print('\nUsing first match. Be more specific if needed.')
            selected_player = matches.iloc[0]
        else:
            selected_player = matches.iloc[0]
    
    if selected_player is not None:
        player_id = selected_player['playerID']
        player_name = selected_player['playerName']
        player_team = selected_player.get('team', 'N/A')
        player_pos = selected_player.get('pos', 'N/A')
        player_salary = selected_player.get('salary', 0)
        
        print('='*60)
        print('SELECTED PLAYER')
        print('='*60)
        print(f'Name: {player_name}')
        print(f'ID: {player_id}')
        print(f'Team: {player_team}')
        print(f'Position: {player_pos}')
        print(f'Salary: ${player_salary:,.0f}')
        print(f'Target Date: {TARGET_DATE}')
        print('='*60)

SELECTED PLAYER
Name: Nikola Jokic
ID: 28908111729
Team: DEN
Position: C
Salary: $12,700
Target Date: 20250210


## Step 3: Load Historical Data for Selected Player

Load player game logs BEFORE the target date (mimics backtest logic).

Loads data for current NBA season + previous season(s) based on NUM_SEASONS parameter.

In [16]:
if selected_player is not None:
    print(f'Loading historical data for {player_name}...')
    print(f'Loading {NUM_SEASONS} seasons of data (current + previous)')
    
    all_historical_data = loader.load_historical_player_logs(
        end_date=TARGET_DATE,
        num_seasons=NUM_SEASONS
    )
    
    print(f'\nTotal games loaded (all players): {len(all_historical_data)}')
    
    player_data = all_historical_data[
        all_historical_data['playerID'] == player_id
    ].copy()
    
    
    if player_data.empty:
        print(f'\nERROR: No historical data found for {player_name}')
        print(f'Player may be new or have no games in database')
    else:
        player_data['gameDate'] = pd.to_datetime(player_data['gameDate'], format='%Y%m%d', errors='coerce')
        player_data = player_data.sort_values('gameDate').reset_index(drop=True)
        
        if 'fpts' not in player_data.columns:
            player_data['fpts'] = player_data.apply(calculate_dk_fantasy_points, axis=1)
        
        numeric_cols = ['pts', 'reb', 'ast', 'stl', 'blk', 'TOV', 'mins', 'fpts']
        for col in numeric_cols:
            player_data[col] = pd.to_numeric(player_data[col], errors='coerce')

        if 'season' not in player_data.columns:
            def get_season(date):
                year = date.year
                if date.month >= 7:
                    return f"{year}-{str(year+1)[-2:]}"
                else:
                    return f"{year-1}-{str(year)[-2:]}"
            player_data['season'] = player_data['gameDate'].apply(get_season)

        player_data = player_data.sort_values(['season', 'gameDate']).reset_index(drop=True)
        
        for col in numeric_cols:
            expanding_mean = player_data.groupby('season')[col].expanding().mean()
            expanding_std = player_data.groupby('season')[col].expanding().std()
            
            player_data[f'{col}_season_avg'] = expanding_mean.reset_index(level=0, drop=True)
            player_data[f'{col}_season_std'] = expanding_std.reset_index(level=0, drop=True)

        print(f'\nHistorical games for {player_name}: {len(player_data)}')
        print(f'Date range: {player_data["gameDate"].min().strftime("%Y-%m-%d")} to {player_data["gameDate"].max().strftime("%Y-%m-%d")}')
        
        current_season_start = loader.get_season_start_date(TARGET_DATE)
        current_season_games = player_data[player_data['gameDate'] >= current_season_start]
        print(f'\nCurrent season (from {current_season_start}): {len(current_season_games)} games')
        print(f'Previous season: {len(player_data) - len(current_season_games)} games')
        
        target_date_dt = pd.to_datetime(TARGET_DATE, format='%Y%m%d')
        if player_data['gameDate'].max() >= target_date_dt:
            print('\n*** WARNING: LOOKAHEAD BIAS DETECTED ***')
            print('Historical data includes target date or later!')
        else:
            days_before = (target_date_dt - player_data['gameDate'].max()).days
            print(f'\nTemporal validation: PASSED')
            print(f'Most recent game is {days_before} days before target date')
        
        print(f'\nLast 5 games (most recent data available for features):')
        display(player_data.tail())

Loading historical data for Nikola Jokic...
Loading 2 seasons of data (current + previous)

Total games loaded (all players): 44208

Historical games for Nikola Jokic: 131
Date range: 2023-10-24 to 2025-02-08

Current season (from 20241001): 47 games
Previous season: 84 games

Temporal validation: PASSED
Most recent game is 2 days before target date

Last 5 games (most recent data available for features):


,playerID,longName,team,teamAbv,teamID,gameID,gameDate,pos,mins,pts,reb,ast,stl,blk,TOV,PF,fga,fgm,fgp,fta,ftm,ftp,tptfga,tptfgm,tptfgp,OffReb,DefReb,fantasyPoints,fantasyPts,plusMinus,usage,tech,created_at,fpts,season,pts_season_avg,pts_season_std,reb_season_avg,reb_season_std,ast_season_avg,ast_season_std,stl_season_avg,stl_season_std,blk_season_avg,blk_season_std,TOV_season_avg,TOV_season_std,mins_season_avg,mins_season_std,fpts_season_avg,fpts_season_std
126,28908111729,Nikola Jokic,DEN,DEN,8,20250201_DEN@CHA,2025-02-01,,37,28,13,17,4,1,4,2,17,9,52.9,9,8,88.9,7,2,28.6,1,12,80.75,,-2,30.04,0,2025-10-06 12:33:01,80.75,2024-25,29.604651,9.227649,12.837209,4.695135,10.302326,3.820589,1.860465,1.264561,0.651163,0.783269,3.186047,1.776243,36.418605,3.983493,66.558140,14.964326
127,28908111729,Nikola Jokic,DEN,DEN,8,20250203_NO@DEN,2025-02-03,,36,27,14,10,1,2,1,2,13,9,69.2,10,8,80.0,3,1,33.3,1,13,67.5,,+11,22.42,0,2025-10-06 12:33:01,68.00,2024-25,29.545455,9.128169,12.863636,4.643530,10.295455,3.776177,1.840909,1.256484,0.681818,0.800370,3.136364,1.786134,36.409091,3.937407,66.590909,14.790896
128,28908111729,Nikola Jokic,DEN,DEN,8,20250205_NO@DEN,2025-02-05,,36,38,8,10,1,1,1,3,28,15,53.6,3,3,100.0,13,5,38.5,2,6,68.0,,+24,34.97,0,2025-10-06 12:33:01,68.00,2024-25,29.733333,9.111431,12.755556,4.647363,10.288889,3.733279,1.822222,1.248433,0.688889,0.792643,3.088889,1.794211,36.400000,3.892884,66.622222,14.623361
129,28908111729,Nikola Jokic,DEN,DEN,8,20250206_ORL@DEN,2025-02-06,,31,28,10,12,2,0,3,2,16,11,68.8,6,4,66.7,4,2,50.0,0,10,61.5,,+22,29.85,0,2025-10-06 12:33:02,64.00,2024-25,29.695652,9.013248,12.695652,4.613360,10.326087,3.700176,1.826087,1.234762,0.673913,0.790340,3.086957,1.774211,36.282609,3.930864,66.565217,14.465134
130,28908111729,Nikola Jokic,DEN,DEN,8,20250208_DEN@PHO,2025-02-08,,29,26,11,9,0,0,2,4,13,11,84.6,3,3,100.0,2,1,50.0,4,7,51.25,,+19,24.67,0,2025-10-06 12:33:02,53.75,2024-25,29.617021,8.931024,12.659574,4.569638,10.297872,3.664844,1.787234,1.249977,0.659574,0.787859,3.063830,1.761968,36.127660,4.030412,66.292553,14.428640


## Step 4: Build Features Using Feature Pipeline

Build features using config-driven FeaturePipeline (matching production backtest):
- Uses YAML configuration from config/features/default_features.yaml
- RollingStatsTransformer: Mean and std for rolling windows [3, 5, 10]
- EWMATransformer: Exponentially weighted moving averages
- TargetTransformer: Create target variable (next game fpts)
- InjuryTransformer: Merge injury data via context and create injury flags

In [ ]:
if selected_player is not None and not player_data.empty:
    print('Building features using FeaturePipeline from config...')
    
    # Load feature configuration
    feature_config = load_feature_config('default_features')
    print(f'Loaded feature config: {feature_config.name}')
    print(f'  Version: {feature_config.version}')
    print(f'  Description: {feature_config.description}')
    print(f'  Stats: {len(feature_config.stats)}')
    print(f'  Transformers: {len(feature_config.transformers)}')
    
    # Build pipeline from config
    pipeline = feature_config.build_pipeline(FeaturePipeline)
    
    print(f'\nFeature pipeline configured with {len(pipeline.transformers)} transformers:')
    for i, transformer in enumerate(pipeline.transformers, 1):
        print(f'  {i}. {transformer.__class__.__name__}')
    
    # Load injury data if available
    injuries_df = None
    try:
        injuries_data = loader.load_slate_data(TARGET_DATE)
        injuries_df = injuries_data.get('injuries', pd.DataFrame())
        if not injuries_df.empty:
            print(f'\nLoaded {len(injuries_df)} injury records for {TARGET_DATE}')
        else:
            print('\nNo injury data available for target date')
    except Exception as e:
        print(f'\nCould not load injury data: {e}')
    
    # Prepare context for pipeline
    context = {}
    if injuries_df is not None and not injuries_df.empty:
        context['injuries'] = injuries_df
    
    # Fit and transform using pipeline
    print(f'\nApplying pipeline to {len(player_data)} games...')
    features_df = pipeline.fit_transform(player_data, context=context)
    
    # Get feature columns (exclude metadata)
    metadata_cols = feature_config.metadata_cols + [
        'playerName', 'longName', 'teamAbv', 'teamID', 'gameID', 
        'PF', 'fga', 'fgm', 'fgp', 'fta', 'ftm', 'ftp', 
        'tptfga', 'tptfgm', 'tptfgp', 'OffReb', 'DefReb', 
        'plusMinus', 'usage', 'tech', 'season', 'injury_date', 
        'injury_return_date', 'injury_description', 'injury_designation',
        'injury_status', 'fantasyPoints', 'fantasyPts', 'created_at', 'updated_at'
    ]
    
    feature_cols = [col for col in features_df.columns 
                    if col not in metadata_cols 
                    and col not in feature_config.stats 
                    and col != 'fpts']
    
    print(f'\nTotal features created: {len(feature_cols)}')
    print(f'\nFeature breakdown:')
    print(f'  Rolling averages: {len([c for c in feature_cols if "_ma" in c])}')
    print(f'  Rolling std devs: {len([c for c in feature_cols if "_std" in c])}')
    print(f'  Rolling min: {len([c for c in feature_cols if "_min" in c])}')
    print(f'  Rolling max: {len([c for c in feature_cols if "_max" in c])}')
    print(f'  EWMA: {len([c for c in feature_cols if "_ewma" in c])}')
    print(f'  Injury features: {len([c for c in feature_cols if "is_" in c])}')
    print(f'  Target: {1 if "target" in feature_cols else 0}')
    
    print(f'\nLast 5 games with features:')
    display(features_df[['gameDate', 'fpts', 'target'] + 
                        [c for c in feature_cols if c.startswith('pts_')][:5]].tail())

In [20]:
display(features_df.head())

display(features_df.tail())

,playerID,longName,team,teamAbv,teamID,gameID,gameDate,pos,mins,pts,reb,ast,stl,blk,TOV,PF,fga,fgm,fgp,fta,ftm,ftp,tptfga,tptfgm,tptfgp,OffReb,DefReb,fantasyPoints,fantasyPts,plusMinus,usage,tech,created_at,fpts,season,pts_season_avg,pts_season_std,reb_season_avg,reb_season_std,ast_season_avg,ast_season_std,stl_season_avg,stl_season_std,blk_season_avg,blk_season_std,TOV_season_avg,TOV_season_std,mins_season_avg,mins_season_std,fpts_season_avg,fpts_season_std,rolling_pts_mean_3,rolling_pts_std_3,rolling_pts_mean_5,rolling_pts_std_5,rolling_pts_mean_10,rolling_pts_std_10,rolling_reb_mean_3,rolling_reb_std_3,rolling_reb_mean_5,rolling_reb_std_5,rolling_reb_mean_10,rolling_reb_std_10,rolling_ast_mean_3,rolling_ast_std_3,rolling_ast_mean_5,rolling_ast_std_5,rolling_ast_mean_10,rolling_ast_std_10,rolling_stl_mean_3,rolling_stl_std_3,rolling_stl_mean_5,rolling_stl_std_5,rolling_stl_mean_10,rolling_stl_std_10,rolling_blk_mean_3,rolling_blk_std_3,rolling_blk_mean_5,rolling_blk_std_5,rolling_blk_mean_10,rolling_blk_std_10,rolling_mins_mean_3,rolling_mins_std_3,rolling_mins_mean_5,rolling_mins_std_5,rolling_mins_mean_10,rolling_mins_std_10,rolling_TOV_mean_3,rolling_TOV_std_3,rolling_TOV_mean_5,rolling_TOV_std_5,rolling_TOV_mean_10,rolling_TOV_std_10,rolling_PF_mean_3,rolling_PF_std_3,rolling_PF_mean_5,rolling_PF_std_5,rolling_PF_mean_10,rolling_PF_std_10,rolling_fga_mean_3,rolling_fga_std_3,rolling_fga_mean_5,rolling_fga_std_5,rolling_fga_mean_10,rolling_fga_std_10,rolling_fgm_mean_3,rolling_fgm_std_3,rolling_fgm_mean_5,rolling_fgm_std_5,rolling_fgm_mean_10,rolling_fgm_std_10,rolling_fgp_mean_3,rolling_fgp_std_3,rolling_fgp_mean_5,rolling_fgp_std_5,rolling_fgp_mean_10,rolling_fgp_std_10,rolling_fta_mean_3,rolling_fta_std_3,rolling_fta_mean_5,rolling_fta_std_5,rolling_fta_mean_10,rolling_fta_std_10,rolling_ftm_mean_3,rolling_ftm_std_3,rolling_ftm_mean_5,rolling_ftm_std_5,rolling_ftm_mean_10,rolling_ftm_std_10,rolling_ftp_mean_3,rolling_ftp_std_3,rolling_ftp_mean_5,rolling_ftp_std_5,rolling_ftp_mean_10,rolling_ftp_std_10,rolling_tptfga_mean_3,rolling_tptfga_std_3,rolling_tptfga_mean_5,rolling_tptfga_std_5,rolling_tptfga_mean_10,rolling_tptfga_std_10,rolling_tptfgm_mean_3,rolling_tptfgm_std_3,rolling_tptfgm_mean_5,rolling_tptfgm_std_5,rolling_tptfgm_mean_10,rolling_tptfgm_std_10,rolling_tptfgp_mean_3,rolling_tptfgp_std_3,rolling_tptfgp_mean_5,rolling_tptfgp_std_5,rolling_tptfgp_mean_10,rolling_tptfgp_std_10,rolling_OffReb_mean_3,rolling_OffReb_std_3,rolling_OffReb_mean_5,rolling_OffReb_std_5,rolling_OffReb_mean_10,rolling_OffReb_std_10,rolling_DefReb_mean_3,rolling_DefReb_std_3,rolling_DefReb_mean_5,rolling_DefReb_std_5,rolling_DefReb_mean_10,rolling_DefReb_std_10,rolling_usage_mean_3,rolling_usage_std_3,rolling_usage_mean_5,rolling_usage_std_5,rolling_usage_mean_10,rolling_usage_std_10,rolling_plusMinus_mean_3,rolling_plusMinus_std_3,rolling_plusMinus_mean_5,rolling_plusMinus_std_5,rolling_plusMinus_mean_10,rolling_plusMinus_std_10,pts_min3,pts_max3,pts_min5,pts_max5,pts_min10,pts_max10,reb_min3,reb_max3,reb_min5,reb_max5,reb_min10,reb_max10,ast_min3,ast_max3,ast_min5,ast_max5,ast_min10,ast_max10,stl_min3,stl_max3,stl_min5,stl_max5,stl_min10,stl_max10,blk_min3,blk_max3,blk_min5,blk_max5,blk_min10,blk_max10,mins_min3,mins_max3,mins_min5,mins_max5,mins_min10,mins_max10,TOV_min3,TOV_max3,TOV_min5,TOV_max5,TOV_min10,TOV_max10,PF_min3,PF_max3,PF_min5,PF_max5,PF_min10,PF_max10,fga_min3,fga_max3,fga_min5,fga_max5,fga_min10,fga_max10,fgm_min3,fgm_max3,fgm_min5,fgm_max5,fgm_min10,fgm_max10,fgp_min3,fgp_max3,fgp_min5,fgp_max5,fgp_min10,fgp_max10,fta_min3,fta_max3,fta_min5,fta_max5,fta_min10,fta_max10,ftm_min3,ftm_max3,ftm_min5,ftm_max5,ftm_min10,ftm_max10,ftp_min3,ftp_max3,ftp_min5,ftp_max5,ftp_min10,ftp_max10,tptfga_min3,tptfga_max3,tptfga_min5,tptfga_max5,tptfga_min10,tptfga_max10,tptfgm_min3,tptfgm_max3,tptfgm_min5,tptfgm_max5,tptfgm_min10,tptfgm_max10,tptfgp_min3,tptfgp_max3,tptfgp_min5,tptfgp_max5,tptfgp_min10,tptfgp_max10,OffReb_min3,OffReb_m

,playerID,longName,team,teamAbv,teamID,gameID,gameDate,pos,mins,pts,reb,ast,stl,blk,TOV,PF,fga,fgm,fgp,fta,ftm,ftp,tptfga,tptfgm,tptfgp,OffReb,DefReb,fantasyPoints,fantasyPts,plusMinus,usage,tech,created_at,fpts,season,pts_season_avg,pts_season_std,reb_season_avg,reb_season_std,ast_season_avg,ast_season_std,stl_season_avg,stl_season_std,blk_season_avg,blk_season_std,TOV_season_avg,TOV_season_std,mins_season_avg,mins_season_std,fpts_season_avg,fpts_season_std,rolling_pts_mean_3,rolling_pts_std_3,rolling_pts_mean_5,rolling_pts_std_5,rolling_pts_mean_10,rolling_pts_std_10,rolling_reb_mean_3,rolling_reb_std_3,rolling_reb_mean_5,rolling_reb_std_5,rolling_reb_mean_10,rolling_reb_std_10,rolling_ast_mean_3,rolling_ast_std_3,rolling_ast_mean_5,rolling_ast_std_5,rolling_ast_mean_10,rolling_ast_std_10,rolling_stl_mean_3,rolling_stl_std_3,rolling_stl_mean_5,rolling_stl_std_5,rolling_stl_mean_10,rolling_stl_std_10,rolling_blk_mean_3,rolling_blk_std_3,rolling_blk_mean_5,rolling_blk_std_5,rolling_blk_mean_10,rolling_blk_std_10,rolling_mins_mean_3,rolling_mins_std_3,rolling_mins_mean_5,rolling_mins_std_5,rolling_mins_mean_10,rolling_mins_std_10,rolling_TOV_mean_3,rolling_TOV_std_3,rolling_TOV_mean_5,rolling_TOV_std_5,rolling_TOV_mean_10,rolling_TOV_std_10,rolling_PF_mean_3,rolling_PF_std_3,rolling_PF_mean_5,rolling_PF_std_5,rolling_PF_mean_10,rolling_PF_std_10,rolling_fga_mean_3,rolling_fga_std_3,rolling_fga_mean_5,rolling_fga_std_5,rolling_fga_mean_10,rolling_fga_std_10,rolling_fgm_mean_3,rolling_fgm_std_3,rolling_fgm_mean_5,rolling_fgm_std_5,rolling_fgm_mean_10,rolling_fgm_std_10,rolling_fgp_mean_3,rolling_fgp_std_3,rolling_fgp_mean_5,rolling_fgp_std_5,rolling_fgp_mean_10,rolling_fgp_std_10,rolling_fta_mean_3,rolling_fta_std_3,rolling_fta_mean_5,rolling_fta_std_5,rolling_fta_mean_10,rolling_fta_std_10,rolling_ftm_mean_3,rolling_ftm_std_3,rolling_ftm_mean_5,rolling_ftm_std_5,rolling_ftm_mean_10,rolling_ftm_std_10,rolling_ftp_mean_3,rolling_ftp_std_3,rolling_ftp_mean_5,rolling_ftp_std_5,rolling_ftp_mean_10,rolling_ftp_std_10,rolling_tptfga_mean_3,rolling_tptfga_std_3,rolling_tptfga_mean_5,rolling_tptfga_std_5,rolling_tptfga_mean_10,rolling_tptfga_std_10,rolling_tptfgm_mean_3,rolling_tptfgm_std_3,rolling_tptfgm_mean_5,rolling_tptfgm_std_5,rolling_tptfgm_mean_10,rolling_tptfgm_std_10,rolling_tptfgp_mean_3,rolling_tptfgp_std_3,rolling_tptfgp_mean_5,rolling_tptfgp_std_5,rolling_tptfgp_mean_10,rolling_tptfgp_std_10,rolling_OffReb_mean_3,rolling_OffReb_std_3,rolling_OffReb_mean_5,rolling_OffReb_std_5,rolling_OffReb_mean_10,rolling_OffReb_std_10,rolling_DefReb_mean_3,rolling_DefReb_std_3,rolling_DefReb_mean_5,rolling_DefReb_std_5,rolling_DefReb_mean_10,rolling_DefReb_std_10,rolling_usage_mean_3,rolling_usage_std_3,rolling_usage_mean_5,rolling_usage_std_5,rolling_usage_mean_10,rolling_usage_std_10,rolling_plusMinus_mean_3,rolling_plusMinus_std_3,rolling_plusMinus_mean_5,rolling_plusMinus_std_5,rolling_plusMinus_mean_10,rolling_plusMinus_std_10,pts_min3,pts_max3,pts_min5,pts_max5,pts_min10,pts_max10,reb_min3,reb_max3,reb_min5,reb_max5,reb_min10,reb_max10,ast_min3,ast_max3,ast_min5,ast_max5,ast_min10,ast_max10,stl_min3,stl_max3,stl_min5,stl_max5,stl_min10,stl_max10,blk_min3,blk_max3,blk_min5,blk_max5,blk_min10,blk_max10,mins_min3,mins_max3,mins_min5,mins_max5,mins_min10,mins_max10,TOV_min3,TOV_max3,TOV_min5,TOV_max5,TOV_min10,TOV_max10,PF_min3,PF_max3,PF_min5,PF_max5,PF_min10,PF_max10,fga_min3,fga_max3,fga_min5,fga_max5,fga_min10,fga_max10,fgm_min3,fgm_max3,fgm_min5,fgm_max5,fgm_min10,fgm_max10,fgp_min3,fgp_max3,fgp_min5,fgp_max5,fgp_min10,fgp_max10,fta_min3,fta_max3,fta_min5,fta_max5,fta_min10,fta_max10,ftm_min3,ftm_max3,ftm_min5,ftm_max5,ftm_min10,ftm_max10,ftp_min3,ftp_max3,ftp_min5,ftp_max5,ftp_min10,ftp_max10,tptfga_min3,tptfga_max3,tptfga_min5,tptfga_max5,tptfga_min10,tptfga_max10,tptfgm_min3,tptfgm_max3,tptfgm_min5,tptfgm_max5,tptfgm_min10,tptfgm_max10,tptfgp_min3,tptfgp_max3,tptfgp_min5,tptfgp_max5,tptfgp_min10,tptfgp_max10,OffReb_min3,OffReb_m

## Basic Statistics Summary

In [ ]:
if selected_player is not None and not player_data.empty:
    # Plot rolling averages for key stats
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    key_stats = ['pts', 'reb', 'ast', 'mins']
    
    for idx, stat in enumerate(key_stats):
        ax = axes[idx // 2, idx % 2]
        
        # Plot actual values
        ax.plot(features_df.index, features_df[stat], 
                label='Actual', linewidth=2, alpha=0.7, marker='o')
        
        # Plot rolling averages
        for window in feature_config.rolling_windows:
            col = f'{stat}_ma{window}'
            if col in features_df.columns:
                ax.plot(features_df.index, features_df[col], 
                       label=f'{window}-game MA', linewidth=1.5, alpha=0.8)
        
        # Plot EWMA
        ewma_col = f'ewma_{stat}_{feature_config.ewma_span}'
        if ewma_col in features_df.columns:
            ax.plot(features_df.index, features_df[ewma_col], 
                   label=f'EWMA (span={feature_config.ewma_span})', linewidth=1.5, 
                   linestyle='--', alpha=0.8)
        
        ax.set_title(f'{player_name} - {stat.upper()} Trends', fontsize=12, fontweight='bold')
        ax.set_xlabel('Game Date')
        ax.set_ylabel(stat.upper())
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Plot feature correlation with target
    print(f'\nFeature Correlation with Target (Next Game FPTS):')
    print('='*80)
    
    # Filter to non-null targets
    valid_data = features_df[features_df['target'].notna()].copy()
    
    if len(valid_data) > 0:
        # Calculate correlations
        correlations = valid_data[feature_cols].corrwith(valid_data['target']).sort_values(ascending=False)
        
        # Display top 20 positive correlations
        print('\nTop 20 Positively Correlated Features:')
        print('-'*80)
        for feat, corr in correlations.head(20).items():
            if feat != 'target':
                print(f'  {feat:50s} {corr:8.4f}')
        
        # Display top 20 negative correlations
        print('\nTop 20 Negatively Correlated Features:')
        print('-'*80)
        for feat, corr in correlations.tail(20).items():
            if feat != 'target':
                print(f'  {feat:50s} {corr:8.4f}')
        
        # Plot top correlations
        fig, ax = plt.subplots(figsize=(12, 8))
        top_corrs = pd.concat([correlations.head(15), correlations.tail(15)])
        top_corrs = top_corrs[top_corrs.index != 'target']
        
        colors = ['green' if x > 0 else 'red' for x in top_corrs.values]
        top_corrs.plot(kind='barh', ax=ax, color=colors, alpha=0.7)
        
        ax.set_title(f'{player_name} - Feature Correlations with Target FPTS', 
                    fontsize=14, fontweight='bold')
        ax.set_xlabel('Correlation Coefficient')
        ax.set_ylabel('Feature')
        ax.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
        ax.grid(True, alpha=0.3, axis='x')
        
        plt.tight_layout()
        plt.show()
    else:
        print('No valid target data available for correlation analysis')

## Feature Visualizations

Visualize feature distributions and correlations.

## Feature Analysis

Analyze the engineered features for the selected player.

In [ ]:
if selected_player is not None and not player_data.empty:
    print(f'Features for {player_name} - Most Recent Game:')
    print('(These features would be used to predict TARGET_DATE performance)')
    print('='*80)
    
    # Get the most recent game's features
    latest_features = features_df[feature_cols].iloc[-1]
    
    # Display by category
    print(f'\\n{"FANTASY POINTS FEATURES":^80}')
    print('-'*80)
    fpts_feats = [(feat, val) for feat, val in latest_features.items() 
                  if feat.startswith('fpts') or feat == 'target']
    if fpts_feats:
        for feature, value in sorted(fpts_feats):
            print(f'  {feature:40s} {value:12.2f}')
    
    print(f'\\n{"POINTS FEATURES":^80}')
    print('-'*80)
    pts_feats = [(feat, val) for feat, val in latest_features.items() 
                 if feat.startswith('pts')]
    if pts_feats:
        for feature, value in sorted(pts_feats):
            print(f'  {feature:40s} {value:12.2f}')
    
    print(f'\\n{"MINUTES FEATURES":^80}')
    print('-'*80)
    mins_feats = [(feat, val) for feat, val in latest_features.items() 
                  if feat.startswith('mins')]
    if mins_feats:
        for feature, value in sorted(mins_feats):
            print(f'  {feature:40s} {value:12.2f}')
    
    print(f'\\n{"ASSISTS FEATURES":^80}')
    print('-'*80)
    ast_feats = [(feat, val) for feat, val in latest_features.items() 
                 if feat.startswith('ast')]
    if ast_feats:
        for feature, value in sorted(ast_feats):
            print(f'  {feature:40s} {value:12.2f}')
    
    print(f'\\n{"REBOUNDS FEATURES":^80}')
    print('-'*80)
    reb_feats = [(feat, val) for feat, val in latest_features.items() 
                 if feat.startswith('reb')]
    if reb_feats:
        for feature, value in sorted(reb_feats):
            print(f'  {feature:40s} {value:12.2f}')
    
    print(f'\\n{"INJURY FEATURES":^80}')
    print('-'*80)
    injury_feats = [(feat, val) for feat, val in latest_features.items() 
                    if 'injury' in feat or feat.startswith('is_')]
    if injury_feats:
        for feature, value in sorted(injury_feats):
            if isinstance(value, (int, float)):
                print(f'  {feature:40s} {value:12.0f}')
            else:
                print(f'  {feature:40s} {str(value):>12s}')
    else:
        print('  No injury features available')
    
    print(f'\\n{"ALL FEATURES SUMMARY":^80}')
    print('-'*80)
    print(f'Total features: {len(latest_features)}')
    print(f'Non-null features: {latest_features.notna().sum()}')
    print(f'Null features: {latest_features.isna().sum()}')
    
    # Create DataFrame for easier viewing
    features_display = pd.DataFrame({
        'Feature': latest_features.index,
        'Value': latest_features.values
    }).sort_values('Feature')
    
    print(f'\\n\\nComplete feature table:')
    display(features_display)